In [11]:
optuna_param_path = "hypsearch/ner/config.json"
hparam_path = "hypsearch/ner/hparams-cnj-focus-v1.json"
study_to_fix = "cnj_ner_focus_v1"

In [21]:
import optuna
import os
import json
from optuna import Trial
from optuna.storages import RetryFailedTrialCallback
from optuna.trial import TrialState
storage = optuna.storages.RDBStorage(
    url='sqlite:////workspace/models/evaluate_lm_tese/db.sqlite3',
    failed_trial_callback=RetryFailedTrialCallback(max_retry=None)
)
old_study = optuna.study.load_study(study_to_fix, storage=storage)

<ipython-input-21-988298360049>:9: ExperimentalWarning: RetryFailedTrialCallback is experimental (supported from v2.8.0). The interface can change in the future.
  failed_trial_callback=RetryFailedTrialCallback(max_retry=None)
<ipython-input-21-988298360049>:11: FutureWarning: load_study(): Please give all values as keyword arguments. See https://github.com/optuna/optuna/issues/3324 for details.
  old_study = optuna.study.load_study(study_to_fix, storage=storage)


In [13]:
old_trials = old_study.get_trials(deepcopy=False)

In [14]:
if optuna_param_path is not None and os.path.isfile(optuna_param_path):
    optuna_config = json.load(open(optuna_param_path))
else:
    optuna_config = {}

if "pruner" in optuna_config:
    pruner_class = getattr(optuna.pruners, optuna_config["pruner"]["type"])
    pruner = pruner_class(**optuna_config["pruner"].get("attributes", {}))
else:
    pruner = None

if "sampler" in optuna_config:
    sampler_class = getattr(optuna.samplers, optuna_config["sampler"]["type"])
    attributes = optuna_config["sampler"].get("attributes", {})
    if optuna_config["sampler"]["type"] == "GridSampler": 
        n_trials = None  
        for hparam in json.load(open(hparam_path)):
            if hparam["type"] != 'categorical':
                raise Exception(f"{hparam['type']} type selected for hyperparameter {hparam['attributes']['name']}, GridSampler only supports categorical hparams")
            attributes[hparam["attributes"]["name"]] = hparam["attributes"]["choices"]            
    sampler = sampler_class(attributes)
else:
        sampler = None

In [ ]:
optuna.study.delete_study(study_to_fix, storage=storage)

In [9]:
correct_trials = []
for trial in old_trials:
    if trial.state == TrialState.COMPLETE:
        correct_trials.append(optuna.trial.create_trial(
            params=trial.params,
            distributions=trial.distributions,
            value=trial.value
        ))
        
new_study = optuna.create_study(
        study_name=study_to_fix,
        direction='maximize',
        storage=storage,
        pruner=pruner,
        sampler=sampler,
        load_if_exists=False
)
new_study.add_trials(correct_trials)

[I 2023-07-29 20:00:01,970] A new study created in RDB with name: cnj_ner_focus_v1_retry
